In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
pd.options.display.max_rows = 999
from ast import literal_eval

In [2]:
def clean(x):
    return literal_eval(x)

In [3]:
accre_jobs_piped = pd.read_csv('../data/accre_jobs_cleaned.csv', converters={'node_numbers': clean})

# Stan 
    Question  1    Which partition nodes show high number of failures 

In [4]:
accre_jobs_piped['failure'] = np.where(accre_jobs_piped['exitcode'] != '0:0',1,0)

In [5]:
cms_jobs_df = accre_jobs_piped[(accre_jobs_piped['user'] == "cmslocal") | (accre_jobs_piped['user'] == "cmspilot")]

In [6]:
#early_exits_df = cms_jobs_df[(cms_jobs_df['used_total_seconds'] < 1920) & (cms_jobs_df['failure']==0)]

In [7]:
#early_exits_df

In [8]:
#early_exits_df['node_index'].value_counts()

In [39]:
accre_jobs_piped[accre_jobs_piped['failure']==1][['exitcode','failure','node_index','node_numbers']]

,exitcode,failure,node_index,node_numbers
696,0:6,1,gpu,"[0048, 0051]"
4280,0:15,1,gpu,"[0053, 0054]"
4281,0:15,1,gpu,"[0043, 0044]"
8629,1:0,1,cn,[1554]
8632,1:0,1,cn,[1544]
...,...,...,...,...
3813535,1:0,1,cn,[1433]
3813639,1:0,1,gpu,[0048]
3814389,1:0,1,cn,"[1123, 1128]"
3814493,1:0,1,cn,"[1207, 1455]"


In [10]:
accre_jobs_failed = accre_jobs_piped[accre_jobs_piped['failure']==1][['exitcode','failure','node_index','node_numbers']]

In [11]:
accre_jobs_failed['node_index'].value_counts()

gpu           1519
cn            1325
ng              33
eval-dell-       7
capra            2
dougherty        1
Name: node_index, dtype: int64

In [12]:
accre_jobs_failed = accre_jobs_failed[accre_jobs_failed.node_index.isin(['gpu','cn'])]

In [35]:
accre_jobs_failed['node_index'].value_counts()

gpu    1519
cn     1325
Name: node_index, dtype: int64

In [40]:
accre_jobs_failed_nodesplit = pd.DataFrame({col: np.repeat(accre_jobs_failed[col].values,accre_jobs_failed['node_numbers'].str.len())
             for col in accre_jobs_failed.columns.drop('node_numbers')}).assign(**{'node_numbers': np.concatenate(accre_jobs_failed['node_numbers'].values)})

In [41]:
accre_jobs_failed_nodesplit['node_numbers'] = accre_jobs_failed_nodesplit['node_numbers'].astype(str)

In [42]:
accre_jobs_failed_nodesplit[accre_jobs_failed_nodesplit['node_index'] == 'cn']

,exitcode,failure,node_index,node_numbers
6,1:0,1,cn,1554
7,1:0,1,cn,1544
13,1:0,1,cn,1460
19,1:0,1,cn,1460
21,1:0,1,cn,1460
...,...,...,...,...
7251,1:0,1,cn,1433
7253,1:0,1,cn,1123
7254,1:0,1,cn,1128
7255,1:0,1,cn,1207


In [43]:
accre_jobs_failed_gpu = accre_jobs_failed_nodesplit[accre_jobs_failed_nodesplit.node_index == 'gpu']

In [17]:
accre_jobs_failed_gpu

,exitcode,failure,node_index,node_numbers
0,0:6,1,gpu,0048
1,0:6,1,gpu,0051
2,0:15,1,gpu,0053
3,0:15,1,gpu,0054
4,0:15,1,gpu,0043
...,...,...,...,...
7241,0:9,1,gpu,0048
7242,0:9,1,gpu,0020
7243,0:9,1,gpu,0020
7252,1:0,1,gpu,0048


In [44]:
accre_jobs_failed_cpu = accre_jobs_failed_nodesplit[accre_jobs_failed_nodesplit.node_index == 'cn']

In [45]:
accre_jobs_failed_cpu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5563 entries, 6 to 7256
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   exitcode      5563 non-null   object
 1   failure       5563 non-null   int32 
 2   node_index    5563 non-null   object
 3   node_numbers  5563 non-null   object
dtypes: int32(1), object(3)
memory usage: 195.6+ KB


In [46]:
gpu_failures = accre_jobs_failed_gpu['node_numbers'].value_counts()
type(gpu_failures)

pandas.core.series.Series

In [47]:
gpu_failures = gpu_failures.to_frame().reset_index()

In [48]:
gpu_failures.columns = ['node_number','total_failures']

In [23]:
gpu_failures

,node_number,total_failures
0,0035,176
1,0019,107
2,0047,105
3,0023,90
4,0013,67
5,0030,60
6,0014,55
7,0015,51
8,0020,50
9,0032,46


In [49]:
cpu_failures = accre_jobs_failed_cpu['node_numbers'].value_counts()
type(cpu_failures)

pandas.core.series.Series

In [50]:
cpu_failures = cpu_failures.to_frame().reset_index()

In [51]:
cpu_failures.columns = ['node_number','total_failures']

In [52]:
cpu_failures

,node_number,total_failures
0,320,34
1,405,33
2,427,30
3,347,30
4,1101,29
5,1449,28
6,1284,28
7,410,27
8,1437,26
9,403,26


In [28]:
cpu_failures_df = pd.DataFrame({'node_number':cpu_failures.index, 'total_failures':cpu_failures.values})
cpu_failures_df

Exception: Data must be 1-dimensional

In [ ]:
cpu_failures_df.describe()

In [ ]:
gpu_failures_df.describe()

In [ ]:
cpu_failures_75 = cpu_failures_df[cpu_failures_df['total_failures']>=14]
cpu_failures_75

In [ ]:
sns.boxplot(x="node_number", y="total_failures", data=cpu_failures)

# Jason --
    Question: 2   CMS Frequency of failure on particular nodes - user names: "cmslocal" and "cmspilot"

# Maggi -- 
    Question: 3 & 4   Calculation of % of actual memory used and mean % for each group